<h1>Import and Funtion</h1>

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier,ConditionalExponentialClassifier,DecisionTreeClassifier,MaxentClassifier
import re, string, random
import pickle
import os.path
from os import path
#from pattern.en import suggest
#from autocorrect import Speller

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)


def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []
    #spell = Speller(lang='en')

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token = reduce_lengthening(token)
        #Spell Check is time consuming
        #token = suggest(token)[0][0]#word lengthening and Spell Check
        #try:
        #    token = spell(token)#Spell Check
        #except:
        #    pass
        
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)



<h2>Load positive and negative sample and create dataset<h2>

In [4]:
if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
#     text = twitter_samples.strings('tweets.20150430-223406.json')
#     tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

#     all_pos_words = get_all_words(positive_cleaned_tokens_list)

#     freq_dist_pos = FreqDist(all_pos_words)
#     print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

   

<h2>Train the data</h2>


In [ ]:
    train_data = dataset[:7000]
    test_data = dataset[7000:]
    
#     ConditionalExponentialClassifier
#     DecisionTreeClassifier
#     MaxentClassifier
#     NaiveBayesClassifier
    

    modelName = 'NaiveBayesClassifier.pickle'
    if path.exists(modelName):
        f = open(modelName, 'rb')
        classifier = pickle.load(f)
        f.close()
    else:
        classifier = ConditionalExponentialClassifier.train(train_data)
        #print(classifier.show_most_informative_features(10))
        f = open(modelName, 'wb')
        pickle.dump(classifier, f)
        f.close()

    
    print("Accuracy is:", classify.accuracy(classifier, test_data))

<h2>Testing</h2>

In [ ]:
    custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
    #custom_tweet = "The world’s largest animal, the blue whale, has been spotted in “unprecedented numbers” in the waters around Antarctica – a sign that the critically endangered mammal could be staging a comeback."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    test_data = dict([token, True] for token in custom_tokens)
    
    print(custom_tweet, classifier.classify(test_data))
    

In [ ]:
test = {}
test['bad'] = True
print(classifier.classify(test))

In [ ]:
#pos
custom_tweet = "President Donald Trump approaches his first big test this week from a position of unusual weakness."
#neu
#custom_tweet = "Trump has the lowest standing in public opinion of any new president in modern history."
#neg
#custom_tweet = "Trump has displayed little interest in the policy itself, casting it as a thankless chore to be done before getting to tax-cut legislation he valuesmore."
custom_tokens = remove_noise(word_tokenize(custom_tweet))
#print(custom_tokens)
ttt = classifier.prob_classify(dict([token, True] for token in custom_tokens))._prob_dict
print(pow(10,ttt['Negative']),pow(10,ttt['Positive']),)

In [ ]:
print(classifier.classify(dict([token, True] for token in custom_tokens)))

In [ ]:
import nltkModel
def classify(prob,range):
    nprob = pow(10,prob['Negative'])
    pprob = pow(10,prob['Positive'])
    diff = pprob - nprob
    if (nprob < range and  pprob < range) or diff < range:
        return 'Neutral'
    elif diff>0:
        return 'Positive'
    else:
        return 'Negative'



classifier =  nltkModel.get_model('')

dict_data = "President Donald Trump approaches his first big test this week from a position of unusual weakness."
test = nltkModel.get_test_dict(dict_data)
prob = classifier.prob_classify(test)._prob_dict
sentiment = classify(prob,0.1)

# Word2Vec

In [5]:
from gensim.models import Word2Vec
from nltk.corpus import gutenberg
emma_vec = Word2Vec(gutenberg.sents('austen-emma.txt'))#read nltk corpus
emma_vec.wv.most_similar('good',  topn=6)

unable to import 'smart_open.gcs', disabling that module


In [239]:
# from nltk.tokenize import sent_tokenize, word_tokenize 
# import warnings
# warnings.filterwarnings(action = 'ignore')   
# import gensim 
# from gensim.models import Word2Vec 

cleaned_tokens = positive_cleaned_tokens_list+negative_cleaned_tokens_list
model = Word2Vec(cleaned_tokens, window=5, size= 20,min_count=1, workers=4)
#model = Word2Vec(gutenberg.sents('austen-emma.txt'))#read nltk corpus


In [240]:
model.wv.most_similar('bad',  topn=6)

[('sick', 0.9933308959007263),
 ('hungry', 0.9926828742027283),
 ('bore', 0.9917027354240417),
 ('excite', 0.991385817527771),
 ('make', 0.9912464022636414),
 ('tire', 0.9910789728164673)]

In [241]:
model.wv.most_similar('good',  topn=6)

[('thing', 0.9885330200195312),
 ('time', 0.9867550134658813),
 ('well', 0.9861913919448853),
 ('look', 0.9856318235397339),
 ('tire', 0.9855085611343384),
 ("i'm", 0.9853665828704834)]

In [242]:
model.wv.similarity('bad', 'good')

0.9829313

In [167]:
x_traing = []
for token in positive_cleaned_tokens_list:
    tmp = [0 for i in range(100)]
    for j in token:
        tmp = list(map(lambda x,y: x+y, tmp,model[j]))
    x_traing.append(tmp)
for token in negative_cleaned_tokens_list:
    tmp = [0 for i in range(100)]
    for j in token:
        tmp = list(map(lambda x,y: x+y, tmp,model[j]))
    x_traing.append(tmp)

<ipython-input-167-002c05e68235>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  tmp = list(map(lambda x,y: x+y, tmp,model[j]))
<ipython-input-167-002c05e68235>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  tmp = list(map(lambda x,y: x+y, tmp,model[j]))


In [199]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm


In [243]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
clf = svm.SVC()
#clf.fit(x_traing[24000:], y_traing[24000:])
clf.fit(x_traing, y_traing)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [170]:
x_traing = []
for token in positive_cleaned_tokens_list:
    tmp = [0 for i in range(100)]
    for j in token:
        tmp = list(map(lambda x,y: x+y, tmp,model[j]))
    x_traing.append(tmp)
for token in negative_cleaned_tokens_list:
    tmp = [0 for i in range(100)]
    for j in token:
        tmp = list(map(lambda x,y: x+y, tmp,model[j]))
    x_traing.append(tmp)

<ipython-input-170-002c05e68235>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  tmp = list(map(lambda x,y: x+y, tmp,model[j]))
<ipython-input-170-002c05e68235>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  tmp = list(map(lambda x,y: x+y, tmp,model[j]))


In [234]:
y_traing= [1 if i < len(positive_cleaned_tokens_list) else 0 for i in range(len(x_traing))]



In [235]:
print(len(y_traing))
print(len(positive_cleaned_tokens_list))
y_traing[24990:25010]


30000
25000


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [88]:
import numpy
x_traing = numpy.asarray(x_traing)

In [246]:
import numpy

def word2ModelVec(tokens):
    res = [0 for i in range(100)]
    for j in tokens:
        try:
            res = list(map(lambda x,y: x+y, res,model[j]))
        except:
            res
    return res
#custom_tweet = "good great "
custom_tweet = "bad sick tire"

custom_tokens = remove_noise(word_tokenize(custom_tweet))
token = word2ModelVec(custom_tokens)
token = numpy.asarray(token)
#print(clf.predict(x_traing[-5:]))
#print(clf.predict(x_traing[0:-1]))

print(clf.predict(token.reshape(-1,20)))



[1]


<ipython-input-246-a1b1478b4d49>:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  res = list(map(lambda x,y: x+y, res,model[j]))


In [190]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
result = pca.fit_transform(x_traing)

In [210]:
from sklearn import svm
X = [[5, 5], [10, 10]]
y = [0, 1]
clf = svm.SVC()
clf.fit(X, y)
clf.predict([[8., 8.]])


array([1])

In [215]:
from gensim.test.utils import common_texts
from gensim.sklearn_api import W2VTransformer
model = W2VTransformer(size=10, min_count=1, seed=1)

# What is the vector representation of the word 'graph'?
wordvecs = model.fit(cleaned_tokens)
wordvecs.transform(['good', 'happy'])


array([[-1.7883407 ,  1.6349937 ,  3.5424354 ,  0.13116063,  0.5731063 ,
         0.54748267,  0.4976517 ,  3.687261  ,  3.1158931 ,  1.35926   ],
       [-0.75917786,  2.1996405 ,  2.176901  , -0.8459641 ,  0.34645498,
         0.3855337 ,  1.335731  ,  2.8891735 ,  2.626511  ,  2.3379211 ]],
      dtype=float32)